# TP1 Report SD-TSIA211

$$
\begin{split}
(Aw)_t & = b_t \\
{\~{x}(t)}^T w_1 + w_0 - y(t) {\~{x}(t)}^T w_2 & = y(t) \\
{\~{x}(t)}^T w_1 + w_0 & = y(t) ({\~{x}(t)}^T w_2 + 1) \\
y(t) & = \frac{w_1^T{\~{x}(t)} + w_0}{w_2^T{\~{x}(t)} + 1}
\end{split}
$$
Remark : $({\~{x}(t)}^T w_1) \in \mathbb{R}$ so ${\~{x}(t)}^T w_1 = w_1^T{\~{x}(t)}$

In [1]:
import numpy as np
import matplotlib.pyplot as plt

# Loading data

data_matrix_train, COP_train, data_matrix_test, COP_test, names = np.load('data_center_data_matrix.npy', allow_pickle=True)

# Constructing matrices for min_w ||A w - b||_2**2

matrix_mean = np.mean(data_matrix_train, axis=0)
M = data_matrix_train - matrix_mean
matrix_std = np.std(M, axis=0)
M = M / matrix_std

A = np.hstack([M, np.ones((M.shape[0],1)), -(M.T * COP_train[:,3]).T])
b = COP_train[:,3]

# Constructing matrices for the test set

M_test = (data_matrix_test - matrix_mean) / matrix_std
A_test = np.hstack([M_test, np.ones((M_test.shape[0],1)), -(M_test.T * COP_test[:,3]).T])
b_test = COP_test[:,3]


# Loading raw data
import pandas as pd
data = pd.read_csv('Raw_Dataset_May.csv')

def name_to_subcategory_and_details(col_name):
    if np.isreal(col_name):
        col_name = names[col_name]
    indices = np.nonzero((data['NAME'] == col_name).values)[0]
    if len(indices) > 0:
        subcategory = data['SUBCATEGORY'].iloc[[indices[0]]].values[0]
        details = data['DETAILS'].iloc[[indices[0]]].values[0]
        return subcategory, details
    else:
        print('unknown name')

In [2]:
from numpy.linalg import lstsq
from sklearn.metrics import mean_squared_error
from numpy.linalg import norm

w = lstsq(A,b)[0]

print(w.shape)

pred = A_test @ w
mse = mean_squared_error(pred, b_test)
print(mse)


C:\Users\hamza\AppData\Local\Temp\ipykernel_14152\1869436412.py:5: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  w = lstsq(A,b)[0]


(1785,)
780.8984793523564


In [3]:
from sklearn.linear_model import Ridge

clf = Ridge(alpha=50)
clf.fit(A,b)
pred = clf.predict(A_test)
mse = mean_squared_error(pred, b_test)
print(mse)

88.04687669738527


$$
\nabla f_1(w) = A^T(Aw - b) + \lambda w
$$

The function is linear so it is convex

In [4]:
l = 100
lipshitz_constant = norm(A)**2 + l
max_stepsize = 2/lipshitz_constant

def f(w):
    return 0.5 * norm((A @ w - b))**2 + l/2 * norm(w)**2

def grad_f(w):
    return A.T @ (A @ w - b) + l * w

def grad_descent(step_size=max_stepsize, n_step=500000):
    w = np.zeros(shape=(1785,))
    for i in range(n_step):
        w = w - step_size * grad_f(w)
        if norm(grad_f(w)) <= 1 :
            return (i+1),w
    
    return n_step,w

step, w_res = grad_descent()
print(step)
print(norm(grad_f(w_res)))
print(f(w_res))


$$
F_2 = \frac{1}{2}||Aw - b||^2 + \lambda ||w||_1
$$

So $f_2(w) = \frac{1}{2}||Aw - b||^2$ and $g_2(w) = \lambda ||w||_1$

In [ ]:
l = 200
lipshitz_constant = norm(A)**2 + l
max_stepsize = 2/lipshitz_constant

def f(w):
    return 0.5 * norm((A @ w - b))**2 + l/2 * norm(w)**2

def prox_g(x):
    return np.sign(x) * (np.abs(x) - max_stepsize)

def grad_f(w):
    return A.T @ (A @ w - b) + l * w

def prox_grad_descent(step_size=max_stepsize, n_step=500000):
    w = np.zeros(shape=(1785,))
    for i in range(n_step):
        if(i%100==0):
            print(i, norm(grad_f(w)))
        w = prox_g(w - step_size * grad_f(w))
        if norm(grad_f(w)) <= 1 :
            return (i+1),w
    
    return n_step,w

step, w_res = prox_grad_descent()
print(step)
print(norm(grad_f(w_res)))
print(f(w_res))

0 120927.59130301421
100 7375.699647306174
200 5563.088966933494
300 4545.356053106323
400 3892.3442986719574
500 3432.762428196519
600 3087.9031118479384
700 2817.0326294560346
800 2596.8830072463375
900 2413.1942698450534
1000 2256.723698806911
1100 2121.248441579946
1200 2002.4047501002522
1300 1897.0605034763066
1400 1802.8621900336664
1500 1718.0223032778727
1600 1641.1485730321153
1700 1571.1366747874304
1800 1507.078695461613
1900 1448.2410713135841
2000 1394.0066859486546
2100 1343.8579118004595
2200 1297.3495153653555
2300 1254.104343348336
2400 1213.7916472106058
2500 1176.1256978715971
2600 1140.8559247494848
2700 1107.7625087996162
2800 1076.6477891870973
2900 1047.3402047859886
3000 1019.689236730609
3100 993.5578757039614
3200 968.8227499232087
3300 945.3709296116153
3400 923.1015020405629
3500 901.9293069397492
3600 881.7721968883554
3700 862.553820874746
3800 844.2104485765993
3900 826.6805532588008
4000 809.9068845951731
4100 793.839384479513
4200 778.431586412776
4300

KeyboardInterrupt: 